In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd 
import os 
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.applications import xception
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

In [2]:
img_height=224
img_width=224
classes=1
batch_size=32
patience=5
ePOCH=100

In [3]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
                                                  '../dataset_cat vs dog/image',
                                                  validation_split=0.3,
                                                  subset="training",
                                                  seed=123,
                                                  image_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  label_mode='binary',
                                                  shuffle=True)
valid_ds=tf.keras.preprocessing.image_dataset_from_directory(
                                                  '../dataset_cat vs dog/image',
                                                  validation_split=0.3,
                                                  subset="validation",
                                                  seed=123,
                                                  image_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  label_mode='binary',
                                                  shuffle=True)
test_ds=tf.keras.preprocessing.image_dataset_from_directory(
                                                  '../dataset_cat vs dog/test',
                                                 # validation_split=0.3,
                                                  # subset="training",
                                                  seed=123,
                                                  image_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  # label_mode='categorical',
                                                  shuffle=False)

Found 25000 files belonging to 2 classes.
Using 17500 files for training.
Found 25000 files belonging to 2 classes.
Using 7500 files for validation.
Found 12500 files belonging to 1 classes.


In [4]:
def conv_bn(x, filters, kernel_size, strides=1, activation=True):
    x = tf.keras.layers.Conv2D(filters=filters, 
                               kernel_size=kernel_size, 
                               strides=strides, 
                               padding='SAME')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if activation:
        x = tf.keras.layers.Activation('relu')(x)
        
    return x

In [5]:
def depthwiseConv_bn(x, kernel_size, strides):

    x = tf.keras.layers.DepthwiseConv2D(kernel_size, 
                                        padding='same', 
                                        strides=strides)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    return x

In [6]:
def sepConv_bn_noskip(x, filters, kernel_size, strides=1):
    
    x = depthwiseConv_bn(x, kernel_size=kernel_size, strides=strides)
    x = conv_bn(x, filters=filters, kernel_size=1, strides=1)
    
    return x

In [7]:
def Squeeze_excitation_layer(x):
    
    inputs = x
    squeeze = inputs.shape[-1]/2
    excitation = inputs.shape[-1]
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(squeeze)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dense(excitation)(x)
    x = tf.keras.layers.Activation('sigmoid')(x)
    x = tf.keras.layers.Reshape((1, 1, excitation))(x)
    x = inputs * x

    return x

In [8]:
def MBConv_idskip(x, filters, kernel_size, strides, t, SE=False):
    
    x_input = x
    
    x = conv_bn(x, filters=x.shape[-1] * t, kernel_size=1, strides=1)
    x = depthwiseConv_bn(x, kernel_size=kernel_size, strides=strides)
    if SE:
        x = Squeeze_excitation_layer(x)
    x = conv_bn(x, filters=filters, kernel_size=1, strides=1, activation=False)
    
    if strides==1 and x.shape[3] == x_input.shape[3]:
        return  tf.keras.layers.add([x_input, x])
    else: 
        return x

In [9]:
def MBConv(x, filters, kernel_size, strides, t, n, SE=False):
    
    x = MBConv_idskip(x, filters, kernel_size, strides, t, SE)
    
    for _ in range(1, n):
        x = MBConv_idskip(x, filters, kernel_size, strides=1, t=t, SE=SE)
        
    return x

In [10]:
def MnasNet(n_classes=1000):
    inputs = tf.keras.layers.Input(shape=(224, 224, 3))
    x = conv_bn(inputs, 32, kernel_size=3, strides=2)
    x = sepConv_bn_noskip(x, filters=16, kernel_size=3, strides=1)
    x = MBConv(x, filters=24, kernel_size=3, strides=2, t=6, n=2)
    x = MBConv(x, filters=40, kernel_size=5, strides=2, t=3, n=3, SE=True)
    x = MBConv(x, filters=80, kernel_size=3, strides=2, t=6, n=4)
    x = MBConv(x, filters=96, kernel_size=3, strides=1, t=6, n=2, SE=True)
    x = MBConv(x, filters=192, kernel_size=5, strides=2, t=6, n=3, SE=True)
    x = MBConv(x, filters=320, kernel_size=3, strides=1, t=6, n=1)
    
    x = conv_bn(x, filters=1152, kernel_size=1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    predictions = tf.keras.layers.Dense(classes, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
    return model

In [11]:

model=MnasNet()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [12]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.BinaryCrossentropy(),  
    metrics=['accuracy'] ) 
tb = tf.keras.callbacks.TensorBoard(log_dir='./logs')
checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath='file_name/cp.h5',
                                                 save_weights_only=False,
                                                 verbose=1,
                                                 monitor='val_accuracy',
                                                 save_best_only=True)
EarlyStopping=tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)
# checkpoint = ModelCheckpoint(filepath='file_name',monitor='val_accuracy',mode='auto' ,save_best_only=True,save_freq='epoch')   
early_stop=tf.keras.callbacks.EarlyStopping(patience=patience, monitor = 'val_accuracy')
callback_lists=[tb,checkpoint,early_stop,EarlyStopping]   
history = model.fit(     
                    train_ds,
                    epochs=ePOCH,     
                    validation_data=valid_ds, 
                    verbose=True,
                    callbacks=[])

Epoch 1/100
547/547 [==============================] - 91s 153ms/step - loss: 0.6191 - accuracy: 0.4983 - val_loss: 0.9993 - val_accuracy: 0.5040
Epoch 2/100
547/547 [==============================] - 82s 150ms/step - loss: 0.5053 - accuracy: 0.4983 - val_loss: 0.6234 - val_accuracy: 0.5040
Epoch 3/100
547/547 [==============================] - 86s 157ms/step - loss: 0.4357 - accuracy: 0.4983 - val_loss: 0.5404 - val_accuracy: 0.5040
Epoch 4/100
547/547 [==============================] - 86s 158ms/step - loss: 0.3744 - accuracy: 0.4983 - val_loss: 0.8779 - val_accuracy: 0.5040
Epoch 5/100
547/547 [==============================] - 86s 157ms/step - loss: 0.3214 - accuracy: 0.4983 - val_loss: 0.4890 - val_accuracy: 0.5040
Epoch 6/100
547/547 [==============================] - 87s 159ms/step - loss: 0.2717 - accuracy: 0.4983 - val_loss: 0.3263 - val_accuracy: 0.5040
Epoch 7/100
547/547 [==============================] - 87s 160ms/step - loss: 0.2378 - accuracy: 0.4983 - val_loss: 0.4355 -